Modules 

In [1]:
import os, sys
import pandas as pd
import numpy as np
import mne
from tqdm import tqdm
from gc import collect as clear
from eelbrain import *
from sklearn.externals import joblib
import time

try:
    if "winsound" not in sys.modules:
        import winsound
    def makeSound(freq = 6000, # Hz
              duration = 3000): # millisecond
        winsound.Beep(freq, duration)
except ImportError:
    if "os" not in sys.modules:
        import winsound
    def makeSound():
        os.system('say -v Amir''s Task finished!')

C:\Users\Amir\Anaconda3\envs\mne\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


Loading files

In [2]:
files = [x[0]+"/"+f for x in os.walk("data/all") for f in x[2] if
               f.endswith(".csv")] 

files = [f for f in files if f[-7:-4] == "plv"]

files = [s for s in files if any(os.path.basename(s)[5:9] in i for i in ["male", "fema", "F010", "F060"])]

#posibleIDs = list(range(120, 155)) + list(range(222, 251)) + list(range(320, 352)) +  list(range(390, 392))

posibleIDs = list(range(120, 155)) + list(range(222, 251)) + list(range(310, 352)) +  list(range(390, 392))


#files = [f for f in files if int(os.path.basename(f)[1:4]) in posibleIDs]


print("I have files", len(files))
print(files[:3])

#files = [f for f in files if os.path.basename(f)[1] != "2"]

grps = np.zeros(len(files))
for i, f in enumerate(files):
    if os.path.basename(f)[1] == "1":
        grps[i] = 1
    if os.path.basename(f)[1] == "2":
        grps[i] = 2
    if os.path.basename(f)[1] == "3":
        grps[i] = 3
    
unique, paradigm = np.unique(grps, return_counts=True)       
print(dict(zip(unique, paradigm / 2)))

male = [s for s in files if any(os.path.basename(s)[5:9] in i for i in ["male"])]
female = [s for s in files if any(os.path.basename(s)[5:9] in i for i in ["fema"])]
baselines = [s for s in files if any(os.path.basename(s)[5:9] in i for i in ["F010"])]
eyes = [s for s in files if any(os.path.basename(s)[5:9] in i for i in ["F060"])]

print("I have males:", len(male),
      "\nI have females:", len(female), 
     "\nI have baseline:", len(baselines), 
     "\nI have eyes:", len(eyes))

I have files 283
['data/all/N101_F010_plv.csv', 'data/all/N101_femal_plv.csv', 'data/all/N101_male_plv.csv']
{1.0: 53.0, 2.0: 34.0, 3.0: 54.5}
I have males: 73 
I have females: 74 
I have baseline: 78 
I have eyes: 58


Removing places were I only have one male/female file

In [3]:
supportfiles = []
for m in male: 
    for f in female: 
        if os.path.basename(m)[1:4] == os.path.basename(f)[1:4]:
            supportfiles.append(m)
            supportfiles.append(f)
            
assert len(supportfiles) % 2 == 0

grps = np.zeros(len(supportfiles))
for i, f in enumerate(supportfiles):
    if os.path.basename(f)[1] == "1" and "_m" in os.path.basename(f):
        grps[i] = 1
    if os.path.basename(f)[1] == "1" and "_f" in os.path.basename(f):
        grps[i] = 2
    if os.path.basename(f)[1] == "2" and "_m" in os.path.basename(f):
        grps[i] = 3
    if os.path.basename(f)[1] == "2" and "_f" in os.path.basename(f):
        grps[i] = 4
    if os.path.basename(f)[1] == "3" and "_m" in os.path.basename(f):
        grps[i] = 5
    if os.path.basename(f)[1] == "3" and "_f" in os.path.basename(f):
        grps[i] = 6
    
unique, paradigm = np.unique(grps, return_counts=True)       
print(dict(zip(unique, paradigm)))

male = [s for s in supportfiles if any(os.path.basename(s)[5:9] in i for i in ["male"])]
female = [s for s in supportfiles if any(os.path.basename(s)[5:9] in i for i in ["fema"])]

{1.0: 26, 2.0: 26, 3.0: 18, 4.0: 18, 5.0: 29, 6.0: 29}


Removing baselines and eyes without support

In [4]:
baselinefiles = []
for b in baselines: 
    for s in supportfiles: 
        if (os.path.basename(b)[1:4] == os.path.basename(s)[1:4]) and (b not in baselinefiles):
            baselinefiles.append(b)

eyesfiles = []
for e in eyes: 
    for s in baselines: 
        if (os.path.basename(e)[1:4] == os.path.basename(s)[1:4]) and (e not in eyesfiles):
            eyesfiles.append(e)
            
print("\nI have baselines:", len(baselinefiles), baselinefiles[:2])
print("\nI have eyesfiles:", len(eyesfiles), eyesfiles[:2])


I have baselines: 73 ['data/all/N101_F010_plv.csv', 'data/all/N121_F010_plv.csv']

I have eyesfiles: 58 ['data/all/N121_F060_plv.csv', 'data/all/N122_F060_plv.csv']


In [5]:
print("I have males:", len(male),
      "\nI have females:", len(female), 
     "\nI have baseline:", len(baselinefiles), 
     "\nI have eyes:", len(eyesfiles))

files = supportfiles + baselinefiles + eyesfiles

print("# files I have:", len(files))

I have males: 73 
I have females: 73 
I have baseline: 73 
I have eyes: 58
# files I have: 277


Setting clusters/areas

In [6]:
chnames = ['Fp1-0', 'Fp2-0', 'F7-0', 'F8-0', 'F3-0', 'F4-0', 'Fz-0', 'FT9-0', 'FT10-0', 'FC5-0', 'FC1-0',
 'FC2-0', 'FC6-0', 'T7-0', 'C3-0', 'Cz-0', 'C4-0', 'T8-0', 'TP9-0', 'CP5-0', 'CP1-0', 'CP2-0',
 'CP6-0', 'TP10-0',  'P7-0', 'P3-0', 'Pz-0', 'P4-0', 'P8-0', 'O1-0', 'O2-0', 'Fp1-1', 'Fp2-1',
 'F7-1', 'F8-1', 'F3-1', 'F4-1', 'Fz-1', 'FT9-1', 'FT10-1', 'FC5-1', 'FC1-1', 'FC2-1', 'FC6-1',
 'T7-1', 'C3-1', 'Cz-1', 'C4-1', 'T8-1', 'TP9-1', 'CP5-1', 'CP1-1', 'CP2-1',  'CP6-1',  'TP10-1',
 'P7-1',  'P3-1',  'Pz-1',  'P4-1',  'P8-1',  'O1-1', 'O2-1']

center = ['Cz-0', 'Fz-0', 'Pz-0']

leftFrontal = ["Fp1-0", "F3-0", "F7-0"]
leftFrontalCentral = ["FC1-0", "FC5-0"]
leftCentralParietal = ["C3-0", "CP1-0", "CP5-0"]
leftParietalOoccipital = ["P3-0", "P7-0", "O1-0"]    
leftTemporal = ["FT9-0", "TP9-0", "T7-0"]

rightFrontal = ["Fp2-0", "F4-0", "F8-0"]
rightFrontalCentral = ["FC2-0", "FC6-0"]
rightCentralParietal = ["C4-0", "CP2-0", "CP6-0"]
rightParietalOoccipital = ["P4-0", "P8-0", "O2-0"]
rightTemporal = ["FT10-0", "TP10-0",  "T8-0"]

roiCh = [leftTemporal, leftParietalOoccipital, leftCentralParietal, leftFrontalCentral, leftFrontal, 
        center, 
        rightFrontal, rightFrontalCentral, rightCentralParietal, rightParietalOoccipital, rightTemporal]

roiNames = ["leftTemporal", "leftParietalOoccipital", "leftCentralParietal", "leftFrontalCentral", "leftFrontal", 
        "center", 
        "rightFrontal", "rightFrontalCentral", "rightCentralParietal", "rightParietalOoccipital", "rightTemporal"]

assert len(roiCh) == len(roiNames), "I don't have the same length of roiCh and roiNames"

roiChange = {}
for i, c in enumerate(roiCh):
    for n in c: 
        roiChange.update({n : roiNames[i]})
        
#pairs = [(ch, ch2) for i, ch in enumerate(roiNames) for ch2 in roiNames]
#labels = [' '.join(pair) for pair in pairs]

pairs = []
for ch1 in roiNames: 
    for ch2 in roiNames: 
        if ch1 == ch2:
            pairs.append((ch1 + "-0", ch2 + "-1"))
labels = [' '.join(pair) for pair in pairs]


In [7]:
connectivity = []
for i, pair1 in enumerate(pairs):
    ch1, ch2 = pair1
    connectivity.append((' '.join(pair1), ' '.join(pair1)))

pairs = set()
for src, dst in connectivity:
    a = labels.index(src)
    b = labels.index(dst)
    pairs.add((a, b))
    
connectivity = np.array(sorted(pairs), np.uint32)
sensor_dim = Categorial("pair", labels, connectivity)

assert len(sensor_dim) == len(labels), "sensor_dim and labels are not at the same length"
print(len(sensor_dim))
print(sensor_dim[:5])

11
Categorial('pair', ('leftTemporal-0 leftTemporal-1', 'leftParietalOoccipital-0 leftParietalOoccipital-1', 'leftCentralParietal-0 leftCentralParietal-1', 'leftFrontalCentral-0 leftFrontalCentral-1', 'leftFrontal-0 leftFrontal-1'))


Building ds - the dataset
**I take only the support files**

In [8]:
%%script false #Not to run
#different variations for paradigms taking
a = [os.path.basename(f)[:4] for f in baselinefiles]
b = [os.path.basename(f)[:4] for f in eyesfiles]
c = [os.path.basename(f)[:4] for f in supportfiles]
filesToTake = list(set(b).intersection(c))

a = [f for f in baselinefiles if os.path.basename(f)[:4] in filesToTake]
b = [f for f in eyesfiles if os.path.basename(f)[:4] in filesToTake]
c = [f for f in supportfiles if os.path.basename(f)[:4] in filesToTake]
filesToTake = b + c


Couldn't find program: 'false'


In [9]:
filesToTake = supportfiles

rows = []
rowsRaw = []
for file in filesToTake:
    items = os.path.basename(file)
    subject = items[1:4]
    condition = items[5:9]
    group = items[1]
    
    df = pd.read_csv(file, usecols=['channel_1', 'channel_2', "alpha", "beta", "gamma"], header=0)
    df[["channel_1","channel_2"]] = df[["channel_1","channel_2"]].replace(list(range(62)), chnames)
    df = df[df['channel_1'].str.strip().str[-1] != df['channel_2'].str.strip().str[-1]].reset_index(drop=True)
    df = df[df['channel_1'].str.strip().str[-1] == "0"]
    df["channel_1"] = df["channel_1"].replace(roiChange.keys(), [ch + "-0" for ch in list(roiChange.values())] )
    df["channel_2"] = df["channel_2"].replace([ch[:-1] + "1" for ch in list(roiChange.keys())], [ch + "-1" for ch in list(roiChange.values())] )
    df = df[df['channel_1'].str.strip().str[:-1] == df['channel_2'].str.strip().str[:-1]].reset_index(drop=True)
    df = df.groupby(["channel_1", "channel_2"]).mean().reset_index()
    df.channel_1 = df.channel_1.astype('category')
    df.channel_1.cat.categories = [ch + "-0" for ch in roiNames]
    df.channel_2 = df.channel_2.astype('category')
    df.channel_2.cat.categories = [ch + "-1" for ch in roiNames]

    df = df.sort_values(by=["channel_1","channel_2"])
    
    rowsRaw.append(df)

    data = {' '.join((ch1, ch2)): (v1, v2, v3) for ch1, ch2, v1, v2, v3 in df.values}
    alpha = NDVar([v[0] for v in list(data.values())], sensor_dim)
    beta = NDVar([v[1] for v in list(data.values())], sensor_dim)
    gamma = NDVar([v[2] for v in list(data.values())], sensor_dim)
    rows.append([subject, condition, group, alpha, beta, gamma])

In [10]:
rowsTake = [r for r in rows if r[0] not in ["135", "171", "304", "152"]]
ds = Dataset.from_caselist(['subject', 'condition', 'group', "alpha", "beta", "gamma"], rowsTake)
ds['subject'].random = True

print(ds.summary())

Key         Type     Values                                                                                          
---------------------------------------------------------------------------------------------------------------------
subject     Factor   101:2, 121:2, 122:2, 123:2, 126:2, 127:2, 128:2, 130:2, 132:2, 133:2, 134:2, 139:2... (69 cells)
condition   Factor   male:69, fema:69                                                                                
group       Factor   1:46, 2:36, 3:56                                                                                
alpha       NDVar    11 pair; 0.0388517 - 0.615851                                                                   
beta        NDVar    11 pair; 0.0496636 - 0.467749                                                                   
gamma       NDVar    11 pair; 0.055003 - 0.440267                                                                    
--------------------------------------------------------

In [11]:
#My acutal analysis with the subset of subjects I want to kickout
#I didn't remove the "pmin" in order to have a stable order of ids in "r.find_clusters"
freq = ["alpha", "beta", "gamma"]
results = []
for f in tqdm(freq): 
    print(f)
    res = testnd.anova(f, 'condition * group * subject(group)',
                       samples = 1000,  ds=ds, pmin=0.01) 
    results.append(res)

makeSound()

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

alpha


 33%|████████████████████████████                                                        | 1/3 [00:20<00:41, 20.75s/it]

beta


 67%|████████████████████████████████████████████████████████                            | 2/3 [00:42<00:21, 21.07s/it]

gamma


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:03<00:00, 21.03s/it]


In [12]:
for f, r in zip(freq, results): 
    print(f)
    print(r.table())

alpha
#   Effect              f_max       p   sig
-------------------------------------------
0   condition            1.12   1.000      
1   group                5.50    .020   *  
2   condition x group    1.62   1.000      
beta
#   Effect              f_max       p   sig
-------------------------------------------
0   condition            0.56   1.000      
1   group                6.09    .015   *  
2   condition x group    1.62   1.000      
gamma
#   Effect              f_max       p   sig
-------------------------------------------
0   condition            0.37   1.000      
1   group                6.46    .017   *  
2   condition x group    0.81   1.000      


In [13]:
myCluster = []
for f, r in zip(freq, results): 
    for i in np.array([i for i in list(r.find_clusters(0.05).values()) if i.name == "id"]).ravel():
        for c, l in zip(np.array(r.cluster(i, 'group')), labels):
            if c != 0:
                myCluster.append([f, l])
                print(l)
print(len(myCluster))
print(myCluster)


rightFrontalCentral-0 rightFrontalCentral-1
rightFrontalCentral-0 rightFrontalCentral-1
leftCentralParietal-0 leftCentralParietal-1
rightFrontalCentral-0 rightFrontalCentral-1
rightTemporal-0 rightTemporal-1
5
[['alpha', 'rightFrontalCentral-0 rightFrontalCentral-1'], ['beta', 'rightFrontalCentral-0 rightFrontalCentral-1'], ['gamma', 'leftCentralParietal-0 leftCentralParietal-1'], ['gamma', 'rightFrontalCentral-0 rightFrontalCentral-1'], ['gamma', 'rightTemporal-0 rightTemporal-1']]


In [14]:
for f, r in zip(freq, results): 
    print(f)
    print(r.find_clusters(0.1))

alpha
id   v        p      sig   effect
---------------------------------
1    5.5014   0.02   *     group 
beta
id   v        p       sig   effect
----------------------------------
1    6.0941   0.015   *     group 
gamma
id   v        p       sig   effect
----------------------------------
1    5.1951   0.045   *     group 
2    5.4425   0.036   *     group 
3    6.4611   0.017   *     group 


Exporting to CSV the aggregated values for each cluster/area

In [15]:
exportData = pd.DataFrame(np.array(ds["subject"]), columns = ["id"])
for f, r in zip(freq, results): 
    for i in np.array([i for i in list(r.find_clusters(0.0499).values()) if i.name == "id"]).ravel():
        for c, l in zip(np.array(r.cluster(i, 'group')), labels):
            if c != 0:
                mask = r.cluster(i, 'group') != 0
                print(f)
                cluster_plv = ds[f].mean(mask)
                colName = f[0] + l.split()[0][:-2]
                exportData[colName] = cluster_plv

exportData.to_csv("exportData.csv")

alpha
beta
gamma
gamma
gamma


Saving plots for each significant cluster/area

In [16]:
plotsFolder = "plots/support vs baseline/" 
plotsFolder = "plots/eyes vs baseline/" 
plotsFolder = "plots/eyes vs baseline vs support/" 
plotsFolder = "plots/eyes vs support/" 
plotsFolder = "plots/temp/" 


plots = []
count = 0
for f, r in zip(freq, results): 
    for i in np.array([i for i in list(r.find_clusters(0.0499).values()) if i.name == "id"]).ravel():
        for c, l in zip(np.array(r.cluster(i, 'group')), labels):
            if c != 0:
                count += 1
                mask = r.cluster(i, 'group') != 0
                print(f)
                cluster_plv = ds[f].mean(mask)
                title = l.split()[0][:-2]
                plot.Barplot(cluster_plv, 'group', ds=ds, show = False, title = title, tight = True).save(plotsFolder + f + title + ".jpg")
count

alpha
Prompt-toolkit does not seem to be supported by the current IPython shell (ZMQInteractiveShell); The Eelbrain GUI needs to block Terminal input to work. Use eelbrain.gui.run() to start GUI interaction.
beta
gamma
gamma
gamma


5